<a href="https://colab.research.google.com/github/bommojuvishu/facebook-prophet-stock/blob/main/Copy_of_fbphrophit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [ ]:
from datetime import date
import yfinance as yf
from fbprophet import Prophet
import operator
import pandas as pd
import io
import traceback

In [ ]:
def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data

In [4]:
def cal_percentage(data):
  x= data['yhat'].tolist()
  y = x[-1] -x[0]
  y=y/x[0]
  y = y *100
  return y

In [5]:
# Calculate the percentage returns
def cal_real_perent(stock, STARTDATE, ENDDATE):
  try:
    data = yf.download(stock +'.NS', STARTDATE, ENDDATE)
    data.reset_index(inplace=True)
    openprice=data['Open'].tolist()[0]
    closeprice = data['Close'].tolist()[-1]
    x= closeprice - openprice
    x= x / openprice
    x = x*100
    return x
  except:
    print("ERROR Real data: ", stock)
    return 0

In [7]:
from google.colab import files
uploaded = files.upload()

Saving nifty50.csv to nifty50.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['nifty50.csv']))
print(df)

In [ ]:
START = "2015-01-01"
TODAY = "2021-11-20"
STARTDATE = TODAY
ENDDATE = "2021-11-30"
# TODAY = date.today().strftime("%Y-%m-%d")
period = 10
result={}

error_stocks=[]
stocks = df['GRASIM'].tolist()

# stocks = ('TTML','HGS', 'INFY')

for stock in stocks:
  try:
    allRts = []
    data = load_data(stock +".NS")
    df_train = data[['Date','Close']]
    df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})
    df_train.tail()
    data_rows = data.shape[0]
    m = Prophet()
    m.fit(df_train)
    future = m.make_future_dataframe(periods=period)
    forecast = m.predict(future)
    predict_rows = forecast.shape[0]
    forecasted = forecast.tail(predict_rows - data_rows)
    forecast_rtr =  cal_percentage(forecasted)
    # Cal Real returns 
    realrtr = cal_real_perent(stock, STARTDATE, ENDDATE)
    
    allRts.append(forecast_rtr)
    allRts.append(realrtr)
    print(allRts)

    result[stock] =allRts

  except Exception as e:
    print("ERROR :", stock  )
    
    error_stocks.append(stock)
    traceback.print_exc()


[*********************100%***********************]  1 of 1 completed


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


[*********************100%***********************]  1 of 1 completed
[3.0649254191283783, -4.727268691890496]
[*********************100%***********************]  1 of 1 completed


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


[*********************100%***********************]  1 of 1 completed
[0.34725872299569416, -5.072302733267614]
[*********************100%***********************]  1 of 1 completed


...

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


[*********************100%***********************]  1 of 1 completed
[1.9218698856068597, -6.31696735296812]
[*********************100%***********************]  1 of 1 completed


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


[*********************100%***********************]  1 of 1 completed
[0.9839683368212853, -7.938223515652927]
[*********************100%***********************]  1 of 1 completed


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
resultdf = pd.DataFrame(result)
resultdf = resultdf.transpose()
resultdf.columns =['predict','real']
print(resultdf.sort_values(by=['predict'], ascending=False))